# Imports

In [1]:
from mmt_multipole_inversion import MagneticSample
from mmt_multipole_inversion import MultipoleInversion
import numpy as np
from pathlib import Path

# Quadrupole

In the following piece of code we generate a perfect quadrupole from two dipoles, using the `MagneticSample` module. We also specify the dimensions of the measurement surface. We save the data in files with the `quadrupole_y-orientation` names.

In [2]:
# Directory to save the files
BASE_DIR = Path('quadrupole_data')
BASE_DIR.mkdir(exist_ok=True)

In [3]:
# Scan height, Scan area x and y, sensor half-legth x and y (meter)
Hz, Sx, Sy, Sdx, Sdy = 2e-6, 20e-6, 20e-6, 0.1e-6, 0.1e-6
Lx, Ly, Lz = Sx * 0.9, Sy * 0.9, 5e-6  # Sample lengths (meter)

# Initialise class to create a sample (with user-defined or random particles)
sample = MagneticSample(Hz, Sx, Sy, Sdx, Sdy, Lx, Ly, Lz)

# Manually set the positions and dipole magnetic moments of two dipoles
Ms = 4.8e5
dipole_positions = np.array([[sample.Lx * 0.5 - 1e-6, sample.Ly * 0.5, -sample.Lz * 0.5],
                             [sample.Lx * 0.5 + 1e-6, sample.Ly * 0.5, -sample.Lz * 0.5]])
mu_s = Ms * (1 * 1e-18) * np.array([[0., 1., 0], [0., -1, 0]])
volumes = np.array([1e-18, 1e-18])
sample.generate_particles_from_array(dipole_positions, mu_s, volumes)

# Generate the dip field: the Bz field flux through the measurement surface
sample.generate_measurement_mesh()

# Redefine positions to make a single particle at the centre (ideal quadrupole)
sample.dipole_positions = np.array(
    [[sample.Lx * 0.5, sample.Ly * 0.5, -sample.Lz * 0.5]])
# Update the N of particles in the internal dict
sample.N_particles = 1

sample.save_data(filename='quadrupole_y-orientation', basedir=BASE_DIR)

Now we proceed to do the numerical inversions using two different basis for the multipole expansion. The fully orthogonal basis is the `spherical_harmonics_basis`, the other basis are for testing purposes.

In [4]:
# Inversions ------------------------------------------------------------------

shinv = MultipoleInversion(BASE_DIR / './MetaDict_quadrupole_y-orientation.json',
                           BASE_DIR / './MagneticSample_quadrupole_y-orientation.npz',
                           expansion_limit='quadrupole',
                           sus_functions_module='spherical_harmonics_basis')
shinv.generate_measurement_mesh()
shinv.generate_forward_matrix()
shinv.compute_inversion(method='sp_pinv')

mcinv = MultipoleInversion(BASE_DIR / './MetaDict_quadrupole_y-orientation.json',
                           BASE_DIR / './MagneticSample_quadrupole_y-orientation.npz',
                           expansion_limit='quadrupole',
                           sus_functions_module='maxwell_cartesian_polynomials')
mcinv.generate_measurement_mesh()
shinv.generate_forward_matrix()
mcinv.compute_inversion(method='sp_pinv')

Parameter Sensor dimensions not found in json file
Setting Sensor dimensions value to ()
Scanning array size = 200 x 200
Scanning array size = 200 x 200
Generation of Q matrix took: 8.7405 s
Using scipy.linalg.pinv for inversion
Parameter Sensor dimensions not found in json file
Setting Sensor dimensions value to ()
Scanning array size = 200 x 200
Scanning array size = 200 x 200
Generation of Q matrix took: 0.2328 s
Generating forward matrix
Generation of Q matrix took: 8.1562 s
Using scipy.linalg.pinv for inversion


Here the stray field signal matrix, the inverted scan signal matrix $\overline{\mathbf{B}}_{z}^{\text{inv}}$ and the residual~\cite{Zhdanov2015} matrix

\begin{equation}
    \overline{\mathbf{B}}_{z}^{\text{res}}=\overline{\mathbf{B}}_{z}^{\text{inv}} - \overline{\mathbf{B}}_{z}
\end{equation}

are computed. The residual error can be quantified by the relative error

\begin{equation}
B_{\text{err}}=\frac{\left\Vert \overline{\mathbf{B}}_{z}^{\text{res}} \right\Vert_{F}}{\left\Vert \overline{\mathbf{B}}_{z}\right\Vert_{F}},
\end{equation}

with $\left\Vert\cdot\right\Vert_{F}$ denoting the Frobenius norm.

In [5]:
sh_invBzNorm = np.linalg.norm(shinv.inv_Bz_array, ord='fro')
sh_BzNorm = np.linalg.norm(shinv.Bz_array, ord='fro')
sh_RelErrResidual = np.linalg.norm(shinv.inv_Bz_array - shinv.Bz_array, ord='fro') / sh_BzNorm

mc_invBzNorm = np.linalg.norm(mcinv.inv_Bz_array, ord='fro')
mc_BzNorm = np.linalg.norm(mcinv.Bz_array, ord='fro')
mc_RelErrResidual = np.linalg.norm(mcinv.inv_Bz_array - mcinv.Bz_array, ord='fro') / mc_BzNorm

In [6]:
print('Computing relative error of inversions Berr using Frobenius norm')

Computing relative error of inversions Berr using Frobenius norm


In [7]:
print(f'Berr using Spherical Harmonics basis: {sh_RelErrResidual:.15f}')

Berr using Spherical Harmonics basis: 0.045167045672727


In [8]:
print(f'Berr using Maxwell-Cartesian polynomials: {mc_RelErrResidual:.15f}')

Berr using Maxwell-Cartesian polynomials: 0.045167045672727
